In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
path = '/content/drive/My Drive/SI670/kaggle1/Trout'
os.chdir(path)
# os.listdir(path)

Mounted at /content/drive


In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

In [3]:
training = pd.read_csv('Trout_training.csv')
testing = pd.read_csv('Trout_testing_features.csv')
training['Date_Time'] = pd.to_datetime(training.Date_Time)
testing['Date_Time'] = pd.to_datetime(testing.Date_Time)


In [24]:
df_training = training.copy()
df_testing = testing.copy()

In [25]:
df_training = df_training.pivot(index='Date_Time',columns='Depth_m')['Water_Temp_C']
df_testing = df_testing.pivot(index='Date_Time',columns='Depth_m')['Water_Temp_C']

In [26]:
df_training.shape,df_testing.shape

((52515, 16), (8760, 15))

In [27]:
print(df_training.columns)
print(df_testing.columns)

Float64Index([ 1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,  8.5,  9.5, 10.5, 11.5,
              12.5, 14.5, 16.5, 18.5, 20.5],
             dtype='float64', name='Depth_m')
Float64Index([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 11.5, 12.5, 14.5,
              16.5, 18.5, 20.5],
             dtype='float64', name='Depth_m')


In [28]:
df_training.head()

Depth_m,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5,12.5,14.5,16.5,18.5,20.5
Date_Time,,,,,,,,,,,,,,,,
2012-04-20 00:00:00,5.257,5.257,5.257,5.205,5.205,5.154,5.231,5.128,5.102,5.076,4.792,4.818,4.766,4.792,4.792,4.818
2012-04-20 01:00:00,5.205,5.231,5.231,5.179,5.205,5.154,5.231,5.128,5.024,4.999,4.792,4.818,4.766,4.792,4.792,4.792
2012-04-20 02:00:00,5.231,5.231,5.231,5.154,5.179,5.128,5.154,4.973,4.895,4.895,4.792,4.792,4.740,4.792,4.792,4.792
2012-04-20 03:00:00,5.205,5.231,5.205,5.154,5.154,5.102,5.128,4.999,4.895,4.921,4.766,4.818,4.766,4.792,4.792,4.766
2012-04-20 04:00:00,5.205,5.231,5.154,5.050,5.050,4.999,5.076,4.999,4.947,4.973,4.818,4.844,4.792,4.792,4.792,4.766


In [29]:
df_testing.isna().sum()

Depth_m
1.5     0
2.5     0
3.5     0
4.5     0
5.5     0
6.5     0
7.5     0
8.5     0
9.5     0
11.5    0
12.5    0
14.5    0
16.5    0
18.5    0
20.5    0
dtype: int64

In [30]:
df_training.dropna(inplace=True)

In [13]:
def minmaxScaler(X_train,X_dev):
    scaler = MinMaxScaler()    
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
    X_dev_scaled = pd.DataFrame(scaler.transform(X_dev),columns=X_dev.columns)
    return X_train_scaled,X_dev_scaled 

def standardScaler(X_train,X_dev):
    scaler = StandardScaler()    
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
    X_dev_scaled = pd.DataFrame(scaler.transform(X_dev),columns=X_dev.columns)
    return X_train_scaled,X_dev_scaled

def lasso(X_train, X_dev, y_train, y_dev):
   
    lasso = Lasso()
    lasso.fit(X_train, y_train)
    prediction = lasso.predict(X_dev)
    lasso_mse = mean_squared_error(y_dev.values, prediction)
    return lasso_mse

In [15]:
def knn(X_train, X_dev, y_train, y_dev):
    knn = KNeighborsRegressor(n_neighbors=5)
    knn.fit(X_train, y_train)
    prediction = knn.predict(X_dev)
    knn_mse = mean_squared_error(y_dev, prediction)
    return knn_mse


def svr(X_train, X_dev, y_train, y_dev):
    svr = SVR()
    svr.fit(X_train, y_train)
    prediction = svr.predict(X_dev)
    svc_mse = mean_squared_error(y_dev, prediction)

    return svc_mse

# XGBRegressor
def xgbr(X_train, X_dev, y_train, y_dev):
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', random_state=0, n_estimators = 50)
    xg_reg.fit(X_train,y_train)
    xg_preds = xg_reg.predict(X_dev)
    xg_mse = mean_squared_error(y_dev, xg_preds)
    return xg_mse

# GradientBoostingRegressor
def gbr(X_train, X_dev, y_train, y_dev):
    gra_reg = GradientBoostingRegressor(random_state=0, n_estimators = 50)
    gra_reg.fit(X_train,y_train)
    gra_preds = gra_reg.predict(X_dev)
    gra_mse = mean_squared_error(y_dev, gra_preds)
    return gra_mse

# AdaBoostRegressor
def ada(X_train, X_dev, y_train, y_dev):
    ada_reg = AdaBoostRegressor(random_state=0, n_estimators = 50)
    ada_reg.fit(X_train,y_train)
    ada_preds = ada_reg.predict(X_dev)
    ada_mse = mean_squared_error(y_dev, ada_preds)
    return ada_mse

def ridge(X_train, X_dev, y_train, y_dev):
   
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    prediction = ridge.predict(X_dev)
    ridge_mse = mean_squared_error(y_dev, prediction)
    return ridge_mse


# Linear regression
def linreg(X_train, X_dev, y_train, y_dev):
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    prediction = linreg.predict(X_dev)
    linreg_mse = mean_squared_error(y_dev, prediction)
    return linreg_mse


# Random Forests
def rf(X_train, X_dev, y_train, y_dev):
    rf = RandomForestRegressor(random_state=0)
    rf.fit(X_train,y_train)
    prediction = rf.predict(X_dev)
    rf_mse = mean_squared_error(y_dev, prediction)
    return rf_mse
    

In [31]:
train_X = df_training.drop(columns=[10.5,9.5])
train_y = df_training[9.5]

test_X = df_testing.drop(columns=[9.5])
test_y = df_testing[9.5]

In [32]:
from sklearn.neural_network import MLPRegressor
train_X,test_X = minmaxScaler(train_X,test_X)
train_X,test_X = standardScaler(train_X,test_X)
# MLP
regr = MLPRegressor(random_state=1, max_iter=700, activation = 'tanh').fit(train_X, train_y)
predictions=regr.predict(test_X)
mean_squared_error(test_y,predictions)

0.02978578552121694

In [18]:
xg_mse = xgbr(train_X,test_X,train_y,test_y)
print('XGBRegressor: ', xg_mse)

gbr_mse = gbr(train_X,test_X,train_y,test_y)
print('GradientBoostingRegressor: ', gbr_mse)

ada_mse = ada(train_X,test_X,train_y,test_y)
print('AdaBoostRegressor: ', ada_mse)

knn_mse = knn(train_X,test_X,train_y,test_y)
print('KNN regression: ', knn_mse)

# run time is very long, give up svr for now
# svr_mse = svr(train_X,test_X,train_y,test_y)
# print('Support Vector Regression: ', svr_mse)

XGBRegressor:  0.07821542483179048
GradientBoostingRegressor:  0.0730329556081098
AdaBoostRegressor:  0.23782995197103626
KNN regression:  0.08837656199086757


In [19]:
lasso_mse = lasso(train_X,test_X,train_y,test_y)
print('Lasso regression: ', lasso_mse)

ridge_mse = ridge(train_X,test_X,train_y,test_y)
print('Ridge regression: ', ridge_mse)

linreg_mse = linreg(train_X,test_X,train_y,test_y)
print('Linear regression: ', linreg_mse)

rf_mse = rf(train_X,test_X,train_y,test_y)
print('Random Forests regression: ', rf_mse)

Lasso regression:  0.6890705133226838
Ridge regression:  0.13848601203313762
Linear regression:  0.13857433207859288
Random Forests regression:  0.044457518262164396


In [33]:
# rf_regression = RandomForestRegressor(random_state=0).fit(train_X,train_y)
# prediction = rf_regression.predict(test_X)

In [34]:
# choose MLP as the best model
regr = MLPRegressor(random_state=1, max_iter=700).fit(train_X, train_y)
prediction= regr.predict(test_X)

In [ ]:
Date_Time = pd.date_range('2018-04-20', periods=8760, freq='H').astype(str)
df_predictions = pd.DataFrame({'Date_Time':Date_Time,
                  'Water_Temp_C':prediction})
df_predictions.Date_Time = df_predictions['Date_Time'].astype(str).tolist()
df_predictions.dtypes

Date_Time        object
Water_Temp_C    float64
dtype: object

In [ ]:
df_predictions.to_csv("./Trout_prediction.csv", index=False)